# Study Helper - Enhanced Agentic AI Application
### Assignment 1: Building a Basic Agentic AI Application

---

## What This Agent Does
This Study Helper agent uses **AI-powered tools** to:
- **Generate quizzes** with real questions using LLM
- **Summarize text** intelligently using LLM
- **Explain concepts** with detailed AI-generated explanations
- **Save study notes** for later reference
- **Retrieve saved notes** to review

The agent maintains memory across interactions and can chain multiple tools together!

---

## Part 1: Setup and Environment Configuration

In [1]:
# Step 1: Install required library
!pip install litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 12.9 MB/s eta 0:00:00


In [ ]:
# Step 2: Set up API key
import os

# OPTION 1: For Groq API (if your key starts with 'gsk_')
os.environ["GROQ_API_KEY"] = "Enter Your API"

# OPTION 2: For OpenAI API (if your key starts with 'sk-')
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

print("✓ API key configured!")

✓ API key configured!


In [3]:
# Step 3: Import necessary libraries
from litellm import completion
from typing import List, Dict
import json
import re

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


---
## Part 2: Core Functions

In [4]:
# Generate response from LLM
def generate_response(messages: List[Dict]) -> str:
    """
    Call LLM to generate a response based on conversation history.

    Args:
        messages: List of message dictionaries with 'role' and 'content'

    Returns:
        String response from the LLM
    """
    try:
        # Check which API key is set
        if os.environ.get("GROQ_API_KEY"):
            # Use Groq with a compatible model
            model = "groq/llama-3.3-70b-versatile"  # Fast and good for this task
        else:
            # Use OpenAI
            model = "openai/gpt-4o"

        response = completion(
            model=model,
            messages=messages,
            max_tokens=1024
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating response: {str(e)}"

print("✓ generate_response function defined!")

✓ generate_response function defined!


In [5]:
# Helper function to extract markdown code blocks
def extract_markdown_block(text: str, block_type: str) -> str:
    """
    Extract content from markdown code blocks.

    Args:
        text: The full text containing markdown blocks
        block_type: The type of block to extract (e.g., 'action', 'json')

    Returns:
        Extracted content from the code block
    """
    pattern = f"```{block_type}\\n(.*?)\\n```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    # If no block type specified, try to find any code block
    pattern = f"```\\n(.*?)\\n```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text

print("✓ extract_markdown_block function defined!")

✓ extract_markdown_block function defined!


In [6]:
# Parse LLM response to extract action
def parse_action(response: str) -> Dict:
    """
    Parse the LLM response into a structured action dictionary.

    Args:
        response: Raw response from LLM

    Returns:
        Dictionary with 'tool_name' and 'args'
    """
    try:
        # Extract the action block from markdown
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)

        # Validate the response has required fields
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {
                "tool_name": "error",
                "args": {"message": "Response must have 'tool_name' and 'args' fields."}
            }
    except json.JSONDecodeError:
        return {
            "tool_name": "error",
            "args": {"message": "Invalid JSON response. Please use proper JSON format."}
        }
    except Exception as e:
        return {
            "tool_name": "error",
            "args": {"message": f"Error parsing response: {str(e)}"}
        }

print("✓ parse_action function defined!")

✓ parse_action function defined!


---
## Part 3: Enhanced AI-Powered Tools

These tools now use the LLM to generate real, intelligent responses!

In [7]:
# Global storage for notes
study_notes = {}

# Tool 1: Generate Quiz (AI-Powered)
def generate_quiz(topic: str, num_questions: int = 5) -> dict:
    """
    Generate a quiz on a given topic using AI.

    Args:
        topic: The subject for the quiz
        num_questions: Number of questions to generate (default: 5)

    Returns:
        Dictionary with AI-generated quiz questions
    """
    try:
        print(f"    Generating {num_questions} quiz questions about {topic}...")

        # Create prompt for quiz generation
        quiz_prompt = [
            {
                "role": "system",
                "content": "You are an expert educational quiz creator."
            },
            {
                "role": "user",
                "content": f"""Create {num_questions} multiple-choice quiz questions about {topic}.

For each question, provide:
1. A clear, educational question
2. Four answer options (A, B, C, D)
3. The correct answer letter
4. A brief explanation

Format each question like this:

Q1: [Question text]
A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]
Correct: [Letter]
Explanation: [Brief explanation]

Make the questions educational and progressively challenging."""
            }
        ]

        # Generate quiz using LLM
        response = generate_response(quiz_prompt)

        return {
            "topic": topic,
            "num_questions": num_questions,
            "quiz_content": response,
            "status": "success",
            "message": f" Generated {num_questions} AI-powered questions about {topic}"
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✓ generate_quiz tool defined (AI-powered)!")

✓ generate_quiz tool defined (AI-powered)!


In [8]:
# Tool 2: Summarize Text (AI-Powered)
def summarize_text(text: str, max_sentences: int = 3) -> dict:
    """
    Summarize a given text using AI.

    Args:
        text: The text to summarize
        max_sentences: Target number of sentences (default: 3)

    Returns:
        Dictionary with AI-generated summary
    """
    try:
        print(f"    Summarizing text (target: {max_sentences} sentences)...")

        # Create prompt for summarization
        summary_prompt = [
            {
                "role": "system",
                "content": "You are an expert at creating concise, clear summaries."
            },
            {
                "role": "user",
                "content": f"""Summarize the following text in approximately {max_sentences} sentences.
Focus on the main ideas and key points.

Text to summarize:
{text}

Provide:
1. A brief summary
2. 3-5 key points as bullet points"""
            }
        ]

        # Generate summary using LLM
        response = generate_response(summary_prompt)

        return {
            "original_length": len(text.split()),
            "summary": response,
            "target_sentences": max_sentences,
            "status": "success",
            "message": " Text summarized successfully"
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✓ summarize_text tool defined (AI-powered)!")

✓ summarize_text tool defined (AI-powered)!


In [9]:
# Tool 3: Explain Concept (AI-Powered)
def explain_concept(concept: str, difficulty: str = "simple") -> dict:
    """
    Explain a concept using AI at different difficulty levels.

    Args:
        concept: The concept to explain
        difficulty: 'simple', 'medium', or 'detailed' (default: 'simple')

    Returns:
        Dictionary with AI-generated explanation
    """
    try:
        print(f"    Explaining '{concept}' at {difficulty} level...")

        # Map difficulty to instruction
        difficulty_map = {
            "simple": "Explain this in simple terms that a beginner can understand. Use analogies and examples.",
            "medium": "Provide a moderate explanation with some technical details and practical examples.",
            "detailed": "Give a comprehensive, detailed explanation including technical aspects, theory, and real-world applications."
        }

        instruction = difficulty_map.get(difficulty, difficulty_map["simple"])

        # Create prompt for explanation
        explain_prompt = [
            {
                "role": "system",
                "content": "You are an expert educator who can explain complex concepts clearly."
            },
            {
                "role": "user",
                "content": f"{instruction}\n\nConcept: {concept}\n\nProvide:\n1. A clear explanation\n2. 2-3 practical examples\n3. Key takeaways"
            }
        ]

        # Generate explanation using LLM
        response = generate_response(explain_prompt)

        return {
            "concept": concept,
            "difficulty_level": difficulty,
            "explanation": response,
            "status": "success",
            "message": f" Explained '{concept}' at {difficulty} level"
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✓ explain_concept tool defined (AI-powered)!")

✓ explain_concept tool defined (AI-powered)!


In [10]:
# Tool 4: Save Notes
def save_notes(title: str, content: str) -> dict:
    """
    Save study notes for later retrieval.

    Args:
        title: Title/name for the notes
        content: The note content

    Returns:
        Dictionary with save status
    """
    try:
        study_notes[title] = content
        return {
            "status": "success",
            "message": f" Notes '{title}' saved successfully!",
            "total_notes": len(study_notes)
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✓ save_notes tool defined!")

✓ save_notes tool defined!


In [11]:
# Tool 5: Retrieve Notes
def retrieve_notes(title: str = None) -> dict:
    """
    Retrieve saved study notes.

    Args:
        title: Specific note title to retrieve (optional)

    Returns:
        Dictionary with notes or list of available notes
    """
    try:
        if title:
            if title in study_notes:
                return {
                    "status": "success",
                    "title": title,
                    "content": study_notes[title],
                    "message": f" Retrieved notes: {title}"
                }
            else:
                return {
                    "status": "error",
                    "message": f" Notes '{title}' not found"
                }
        else:
            return {
                "status": "success",
                "available_notes": list(study_notes.keys()),
                "total": len(study_notes),
                "message": f" Found {len(study_notes)} saved notes"
            }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("✓ retrieve_notes tool defined!")

✓ retrieve_notes tool defined!


---
## Part 4: Agent Behavior Definition

In [12]:
# Define agent rules and behavior
agent_rules = [{
    "role": "system",
    "content": """
You are a Study Helper AI agent designed to assist students with their learning.

Available tools (ALL POWERED BY AI):

1. generate_quiz(topic: str, num_questions: int) -> dict
   - Creates AI-generated quiz with real questions
   - num_questions: number of questions (default: 5)

2. summarize_text(text: str, max_sentences: int) -> dict
   - AI-powered text summarization with key points
   - max_sentences: target summary length (default: 3)

3. explain_concept(concept: str, difficulty: str) -> dict
   - AI explains concepts with examples
   - difficulty: 'simple', 'medium', or 'detailed' (default: 'simple')

4. save_notes(title: str, content: str) -> dict
   - Saves study notes for later retrieval

5. retrieve_notes(title: str) -> dict
   - Retrieves saved notes by title
   - If no title provided, lists all available notes

6. terminate(message: str)
   - Ends the conversation with a summary

IMPORTANT RULES:
- Always respond with exactly ONE action per turn
- Choose the most appropriate tool for the user's request
- If the user wants to end the conversation, use terminate
- Be helpful and encouraging to students
- All tools use AI to generate high-quality responses

Response format (you MUST follow this exactly):
```action
{
  "tool_name": "name_of_tool",
  "args": {"arg1": "value1", "arg2": "value2"}
}
```

Example:
User: "Create a quiz about Python"
```action
{
  "tool_name": "generate_quiz",
  "args": {"topic": "Python programming", "num_questions": 5}
}
```
"""
}]

print("✓ Agent rules defined!")

✓ Agent rules defined!


---
## Part 5: Agent Loop Implementation

In [13]:
def run_agent(max_iterations: int = 10):
    """
    Main agent loop that processes user requests.

    Args:
        max_iterations: Maximum number of interactions allowed
    """
    # Initialize memory with agent rules
    memory = agent_rules.copy()
    iterations = 0

    print("\n" + "="*60)
    print(" Study Helper Agent Started! (AI-POWERED)")
    print("="*60)
    print("I can help you with:")
    print("  • Creating AI-generated quizzes")
    print("  • Summarizing text with AI")
    print("  • Explaining concepts intelligently")
    print("  • Saving and retrieving notes")
    print("\nAll tools are powered by AI for better results!")
    print("Type 'quit' or 'exit' to end the session.")
    print("="*60 + "\n")

    while iterations < max_iterations:
        # Get user input
        user_input = input("\n You: ").strip()

        # Check for exit commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            user_input = "Please terminate the session with a nice goodbye message."

        if not user_input:
            print("  Please enter a message.")
            continue

        # Add user message to memory
        memory.append({"role": "user", "content": user_input})

        # Generate response from LLM
        print("\n Agent: Thinking...")
        response = generate_response(memory)
        print(f"\n Agent Response:\n{response}")

        # Parse the action from response
        action = parse_action(response)
        print(f"\n  Action: {action['tool_name']}")

        # Execute the appropriate tool
        result = None

        try:
            if action["tool_name"] == "generate_quiz":
                result = generate_quiz(
                    topic=action["args"].get("topic", ""),
                    num_questions=action["args"].get("num_questions", 5)
                )

            elif action["tool_name"] == "summarize_text":
                result = summarize_text(
                    text=action["args"].get("text", ""),
                    max_sentences=action["args"].get("max_sentences", 3)
                )

            elif action["tool_name"] == "explain_concept":
                result = explain_concept(
                    concept=action["args"].get("concept", ""),
                    difficulty=action["args"].get("difficulty", "simple")
                )

            elif action["tool_name"] == "save_notes":
                result = save_notes(
                    title=action["args"].get("title", ""),
                    content=action["args"].get("content", "")
                )

            elif action["tool_name"] == "retrieve_notes":
                result = retrieve_notes(
                    title=action["args"].get("title")
                )

            elif action["tool_name"] == "terminate":
                print(f"\n {action['args'].get('message', 'Session ended.')}")
                print("\n" + "="*60)
                print("Thanks for using AI-Powered Study Helper! ")
                print("="*60)
                break

            elif action["tool_name"] == "error":
                result = {"error": action["args"]["message"]}

            else:
                result = {"error": f"Unknown tool: {action['tool_name']}"}

        except Exception as e:
            result = {"error": f"Error executing tool: {str(e)}"}

        # Display result
        if result:
            print(f"\n Result:")
            print("="*60)
            # Pretty print the result
            if isinstance(result, dict):
                for key, value in result.items():
                    if key not in ['status']:
                        if isinstance(value, str) and len(value) > 200:
                            print(f"{key}:\n{value}\n")
                        else:
                            print(f"{key}: {value}")
            print("="*60)

            # Update memory with assistant response and tool result
            memory.append({"role": "assistant", "content": response})
            memory.append({"role": "user", "content": json.dumps(result)})

        iterations += 1

    if iterations >= max_iterations:
        print("\n  Maximum iterations reached. Session ended.")

print("✓ run_agent function defined!")

✓ run_agent function defined!


---
## Part 6: Run the Agent

Now let's run the AI-Powered Study Helper agent!

In [14]:
# Run the agent
run_agent(max_iterations=10)


 Study Helper Agent Started! (AI-POWERED)
I can help you with:
  • Creating AI-generated quizzes
  • Summarizing text with AI
  • Explaining concepts intelligently
  • Saving and retrieving notes

All tools are powered by AI for better results!
Type 'quit' or 'exit' to end the session.


 You: Create a quiz about Neural Networks with 3 questions

 Agent: Thinking...

 Agent Response:
```action
{
  "tool_name": "generate_quiz",
  "args": {"topic": "Neural Networks", "num_questions": 3}
}
```

  Action: generate_quiz
    Generating 3 quiz questions about Neural Networks...

 Result:
topic: Neural Networks
num_questions: 3
quiz_content:
Q1: What is the primary function of the activation function in a neural network?
A) To reduce the dimensionality of the input data
B) To increase the complexity of the model
C) To introduce non-linearity into the model, allowing it to learn and represent more complex relationships
D) To decrease the learning rate of the model
Correct: C
Explanation: The a

---
## Example Interactions

### Example 1: AI-Generated Quiz
```
You: Create a quiz about Neural Networks with 3 questions
Agent: [Generates real quiz questions using AI]
Result: Shows actual multiple-choice questions with explanations
```

### Example 2: AI Concept Explanation
```
You: Explain recursion in detail
Agent: [Uses AI to explain with examples]
Result: Detailed explanation with practical examples
```

### Example 3: AI Text Summarization
```
You: Summarize this: "Machine learning is a subset of artificial intelligence..."
Agent: [AI analyzes and summarizes]
Result: Concise summary with key points
```

### Example 4: Memory Chain
```
You: Explain machine learning in simple terms
Agent: [AI explains]
You: Now create a quiz about what you just explained
Agent: [AI creates quiz based on previous explanation]
```

### Example 5: Save and Retrieve
```
You: Create a quiz about Python
Agent: [Generates quiz]
You: Save this quiz as "Python Basics"
Agent: [Saves the quiz]
You: Show me all my saved notes
Agent: [Lists all saved notes]
```

---

##  Key Features

### AI-Powered Tools:
✅ **Smart Quiz Generation** - Creates real multiple-choice questions with explanations

✅ **Intelligent Summarization** - AI analyzes and extracts key points

✅ **Adaptive Explanations** - Adjusts complexity based on difficulty level

✅ **Memory Persistence** - Save and retrieve study materials

✅ **Context Awareness** - Remembers conversation history

### Technical Features:
✅ Comprehensive error handling

✅ JSON parsing with validation

✅ Agent loop with termination

✅ Multi-API support (Groq/OpenAI)

✅ Interactive CLI interface


---

## How to Use

1. **Setup**: Add your Groq API key
2. **Run**: Execute all cells in order
3. **Interact**: Type your requests naturally
4. **Exit**: Type 'quit' or 'exit' when done
